In [72]:
import scrapy
import time
from scrapy_selenium import SeleniumRequest
from scrapy.selector import Selector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from shutil import which
from openpyxl import load_workbook

wb = load_workbook('BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

lista_cnpj_override = "01.608.573/0001-65"

# webdriver.ChromeOptions é usado para definir a preferência para o browser do Chrome
options = webdriver.ChromeOptions()

#faz com que nao abra a página.
options.add_argument("--headless=new")

driver = webdriver.Chrome(executable_path=which('chromedriver'), options=options)

url='https://cvmweb.cvm.gov.br/swb/default.asp?sg_sistema=fundosreg'

driver.get(url)

#pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
WebDriverWait(driver, 10).until(
EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

search_input = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
)
search_input.clear()
#search_input.send_keys(lista_cnpj[0])
search_input.send_keys(lista_cnpj_override)
search_input.send_keys(Keys.ENTER)

find_name_to_store = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
)
stored_name = find_name_to_store.text
find_name_to_store.click()

driver.switch_to.default_content()

#pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

dados_diarios = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
)
dados_diarios.click()

driver.switch_to.default_content()

#pop-up com mensagem irrelevante aparece e precisa ser fechado.
try:
    alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
    alert.accept()
except:
    pass

WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
))

dados_diarios_list = []
for i, _ in enumerate(dropdown_data_pesquisa.options):
    driver.switch_to.default_content()

    WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))
    dropdown_data_pesquisa.select_by_index(i)

    name_selected_dropdown = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))
    name_selected_dropdown = name_selected_dropdown.first_selected_option.get_attribute("value")
    
    linhas_tabela = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//table[@id='dgDocDiario']//tr[position()>1]"))
    )
    linhas_validas = [linha for linha in linhas_tabela if linha.find_element(By.XPATH, "td[2]").text.strip()] 
    
    if linhas_validas:
        ultima_linha = linhas_validas[-1]
        dados = ultima_linha.find_elements(By.XPATH, "td")
        dados_diarios = {
            "Mês": name_selected_dropdown,
            "Dia": dados[0].text.strip(),
            "Quota": dados[1].text.strip(),
            #"Captação no Dia": dados[2].text.strip(),
            #"Resgate no Dia": dados[3].text.strip(),
            "Patrimônio Líquido": dados[4].text.strip(),
            #"Total da Carteira": dados[5].text.strip(),
            "Número de Cotistas": dados[6].text.strip(),
            #"Data da Próxima Informação do PL": dados[7].text.strip(),
        }
        dados_diarios_list.append(dados_diarios)
#dropdown.select_by_visible_text('02/2025')

UnexpectedAlertPresentException: Alert Text: Estes documentos são referentes ao participante anterior a adaptação a Instrução CVM nº 409: BB TOP RENDA FIXA CURTO PRAZO FUNDO DE INVESTIMENTO.
Message: unexpected alert open: {Alert text : Estes documentos são referentes ao participante anterior a adaptação a Instrução CVM nº 409: BB TOP RENDA FIXA CURTO PRAZO FUNDO DE INVESTIMENTO.}
  (Session info: chrome=133.0.6943.98)


In [73]:
dados_diarios_list

[{'Mês': '02/2025',
  'Dia': '13',
  'Quota': '22,253360922',
  'Patrimônio Líquido': '78.614.539.897,52',
  'Número de Cotistas': '45'},
 {'Mês': '01/2025',
  'Dia': '31',
  'Quota': '22,157352602',
  'Patrimônio Líquido': '71.449.766.504,84',
  'Número de Cotistas': '45'},
 {'Mês': '12/2024',
  'Dia': '31',
  'Quota': '21,940251028',
  'Patrimônio Líquido': '67.427.765.365,53',
  'Número de Cotistas': '43'},
 {'Mês': '11/2024',
  'Dia': '29',
  'Quota': '21,742233732',
  'Patrimônio Líquido': '75.725.438.533,68',
  'Número de Cotistas': '43'},
 {'Mês': '10/2024',
  'Dia': '31',
  'Quota': '21,574232528',
  'Patrimônio Líquido': '74.355.450.055,35',
  'Número de Cotistas': '43'},
 {'Mês': '09/2024',
  'Dia': '30',
  'Quota': '21,37853949',
  'Patrimônio Líquido': '80.501.540.372,42',
  'Número de Cotistas': '43'},
 {'Mês': '08/2024',
  'Dia': '30',
  'Quota': '21,204128989',
  'Patrimônio Líquido': '82.715.108.105,03',
  'Número de Cotistas': '43'},
 {'Mês': '07/2024',
  'Dia': '31',


In [ ]:
driver.switch_to.default_content()

WebDriverWait(driver, 10).until(
EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
))
dropdown_data_pesquisa.select_by_index(i)

In [56]:
a = dropdown_data_pesquisa.select_by_index(i)

In [ ]:
dropdown_data_pesquisa.

AttributeError: 'NoneType' object has no attribute 'text'

In [33]:
driver.switch_to.default_content()

WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
))

dropdown_data_pesquisa.select_by_value("12/2024")

In [35]:
driver.switch_to.default_content()

WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
)

search_selections = Select(WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
))

dates = search_selections.options

In [34]:
dates = search_selections.options

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.98)


In [38]:

for i, date in enumerate(dates):
    driver.switch_to.default_content()

    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))

    dropdown_data_pesquisa.select_by_index(i)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.98)
